# Write more functions for GTFS classes

Sunday, March 24, 2019

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

import os
import gtfs_transformer as gt

from shapely.geometry import Point
from shapely.ops import nearest_points
from geopy.distance import great_circle
import matplotlib.pyplot as plt
from tqdm import tqdm as tqdm_base, tqdm_notebook as tqdm

tqdm_base.pandas()

In [131]:
import copy

## Create GTFS objects for July 2018 and August 2018 by reading from directory

### To Do:

I'd like to add the ability to initialize the gtfs object directly from the `.zip` file downloaded from King County Metro, rather than having to unzip the file first. I don't think you can do this directly using [`pandas.read_csv`](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) because the `.zip` file contains multiple `.csv`'s. However, I think it should be possible using the Python [`zipfile`](https://pymotw.com/2/zipfile/) module.

In [117]:
#Initializes GTFS object with tables in given directories, inferring date from directory name
gtfs_jul18 = gt.StaticGTFS('../data/source/gtfs_20180718')
gtfs_aug18 = gt.StaticGTFS('../data/source/gtfs_20180815')

In [10]:
gtfs_aug18.table_names

['agency',
 'block',
 'block_trip',
 'calendar',
 'calendar_dates',
 'fare_attributes',
 'fare_rules',
 'routes',
 'shapes',
 'stop_times',
 'stops',
 'trips']

## Get route id's using `route_ids_from_names` function

Wed 3/27/19: The idea was for this to be a convenient mapping from route names to route ids, but it turns out it's not that convenient to use with DataFrames because you still have to select the `route_id` column.

I wrote a new function called `route_ids()` that returns a Series object instead (by simply selecting the `route_id` column from this DataFrame), which can be used to directly index another DataFrame that's indexed by route id's. I'm planning to "deprecate" the `route_ids_from_names()` function in favor of the `route_ids()` function, assuming the latter works out well after I get a chance to test it more.

In [ ]:
# def route_ids_from_names(self, *route_short_names):
#     if len(route_short_names) == 0: #Get all the id's if none were passed
#         route_short_names = self.routes.route_short_name
#     else:
#         route_short_names = [str(name) for name in route_short_names]

#     return self.routes.loc[self.routes.route_short_name.isin(route_short_names),
#             ['route_short_name', 'route_id']].set_index('route_short_name')

In [13]:
# This creates a mapping for all routes in the routes table
route_ids = gtfs_aug18.route_ids_from_names()
route_ids.head(10)

,route_id
route_short_name,
1,100001
10,100002
101,100003
105,100004
106,100005
107,100006
11,100009
111,100011
113,100012


In [8]:
route_ids.loc['D Line']

route_id    102581
Name: D Line, dtype: int64

In [9]:
route_ids.loc[['10', 'D Line']]

,route_id
route_short_name,
10,100002
D Line,102581


In [12]:
# Can get same table by passing specific lines to function
gtfs_aug18.route_ids_from_names(10, 'D Line')

,route_id
route_short_name,
10,100002
D Line,102581


In [137]:
route_ids.at['10', 'route_id']

100002

In [139]:
# # This doesn't work...
# route_ids.at[['10', 'D Line'], 'route_id']

In [140]:
gtfs_aug18.route_ids_from_names(10, 'D Line').values

array([[100002],
       [102581]])

In [142]:
route_ids.head().to_dict()

{'route_id': {'1': 100001,
  '10': 100002,
  '101': 100003,
  '105': 100004,
  '106': 100005}}

In [144]:
route_ids.to_dict()['route_id']['10']

100002

In [21]:
route_ids.shape

(221, 1)

In [113]:
gtfs_aug18.routes.head(3)

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
0,100001,KCM,1,NaN,Kinnear - Downtown Seattle,3,http://metro.kingcounty.gov/schedules/001/n0.html,NaN,NaN
1,100002,KCM,10,NaN,Capitol Hill - Downtown Seattle,3,http://metro.kingcounty.gov/schedules/010/n0.html,NaN,NaN
2,100003,KCM,101,NaN,Renton Transit Center - Downtown Seattle,3,http://metro.kingcounty.gov/schedules/101/n0.html,NaN,NaN


## We want to display trip headsign and `direction_id` for a route to know which direction is which

However, there may be more than one headsign for a given direction...

In [11]:
gtfs_aug18.trips.head()

,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,peak_flag,fare_id
0,100227,19845,34734386,Downtown Seattle,LOCAL,1,5045281,20047002,0,101.0
1,100227,19845,34734387,Downtown Seattle,LOCAL,1,5045281,20047002,0,101.0
2,100227,19845,34734388,Downtown Seattle,LOCAL,1,5045281,20047002,0,101.0
3,100227,19845,34734389,Downtown Seattle,LOCAL,1,5045281,20047002,0,101.0
4,100227,19845,34734390,Downtown Seattle,LOCAL,1,5045281,20047002,0,101.0


In [14]:
gtfs_aug18.trips.shape

(59857, 10)

In [150]:
gtfs_aug18.trips['trip_short_name'].unique()

array(['LOCAL', 'EXPRESS'], dtype=object)

In [151]:
# Less than 10% of trips are EXPRESS
gtfs_aug18.trips[gtfs_aug18.trips['trip_short_name']=='EXPRESS'].shape

(5195, 10)

### Try out different ways of aggregating the `trips` data

In [20]:
gtfs_aug18.trips.groupby(by=['route_id', 'trip_headsign']).agg({'direction_id': 'mean'}).shape

(542, 1)

In [26]:
gtfs_aug18.trips.groupby(by=['route_id', 'direction_id']).agg({'trip_headsign': 'max'}).shape

(438, 1)

In [28]:
gtfs_aug18.trips.groupby(by=['route_id', 'trip_headsign']).agg({'direction_id': 'max', 'shape_id': 'max'}).head(10)

direction_id  shape_id
route_id trip_headsign                                                    
100001   Downtown Seattle                                      1  20001010
         Kinnear Seattle Center W                              0  11001012
100002   Atlantic Base Via Broadway                            1  20010004
         Capitol Hill Via 15th Ave E                           0  11010010
         Downtown Seattle                                      1  20010007
100003   Downtown Seattle                                      1  10101008
         S Renton P&R                                          0  21101007
100004   Renton Highlands                                      0  11105002
         Renton Renton Tech Coll                               1  20105003
100005   International District Chinatown Station              1  10106037

In [24]:
gtfs_aug18.routes.shape

(221, 9)

In [25]:
gtfs_aug18.trips.direction_id.unique()

array([1, 0])

In [33]:
gtfs_aug18.trips.groupby(
    by=['route_id', 'trip_headsign', 'direction_id', 'shape_id']
).agg({'trip_id': ['count', 'min', 'max']}).head(20)

trip_id  \
                                                                                    count   
route_id trip_headsign                                      direction_id shape_id           
100001   Downtown Seattle                                   1            20001009       8   
                                                                         20001010     274   
         Kinnear Seattle Center W                           0            11001011     270   
                                                                         11001012      12   
100002   Atlantic Base Via Broadway                         1            20010004      10   
         Capitol Hill Via 15th Ave E                        0            11010005       8   
                                                                         11010006     470   
                                                                         11010010       8   
         Downtown Seattle                                   1            20010002     470   
                                                                         20010007       2   
100003   Downtown Seattle                                   1            10101006     114   
                                                                         10101008     114   
         S Renton P&R                                       0            21101005     121   
                                                                         21101007     121   
100004   Renton Highlands                                   0            11105002     168   
         Renton Renton Tech Coll                            1            20105003     174   
100005   International District Chinatown Station           1            10106037       6   
         International District Chinatown Station Rainie... 1            10106034     321   
         Renton Rainier Beach Station                       0            21106108     334   
100006   Beacon Hill Station Rainier Beach Station          1            10107018     246   

                                                                                             \
                                                                                        min   
route_id trip_headsign                                      direction_id shape_id             
100001   Downtown Seattle                                   1            20001009  34736146   
                                                                         20001010  34735996   
         Kinnear Seattle Center W                           0            11001011  34735973   
                                                                         11001012  34736010   
100002   Atlantic Base Via Broadway                         1            20010004  34740661   
         Capitol Hill Via 15th Ave E                        0            11010005  34735719   
                                                                         11010006  38426627   
                                                                         11010010  38426760   
         Downtown Seattle                                   1            20010002  34735660   
                                                                         20010007  34740670   
100003   Downtown Seattle                                   1            10101006  39170852   
                                                                         10101008  39566869   
         S Renton P&R                                       0            21101005  39170811   
                                                                         21101007  39566879   
100004   Renton Highlands                                   0            11105002  39169481   
         Renton Renton Tech Coll                            1            20105003  39169479   
100005   International District Chinatown Station           1            10106037  34816202   
         International District Chinatown 

In [30]:
gtfs_aug18.trips.groupby(
    by=['route_id', 'trip_headsign', 'shape_id', 'direction_id']
).agg({'trip_id': 'count'}).shape

(914, 1)

In [92]:
gtfs_aug18.trips.merge(gtfs_aug18.routes[['route_id', 'route_short_name', 'route_desc']], on='route_id').groupby(
    by=['route_id', 'route_short_name', 'route_desc', 'trip_headsign', 'direction_id', 'shape_id']
).agg({'trip_id': ['count', 'min', 'max']}).head(20)

trip_id  \
                                                                                                                                                       count   
route_id route_short_name route_desc                                        trip_headsign                                      direction_id shape_id           
100001   1                Kinnear - Downtown Seattle                        Downtown Seattle                                   1            20001009       8   
                                                                                                                                            20001010     274   
                                                                            Kinnear Seattle Center W                           0            11001011     270   
                                                                                                                                            11001012      12   
100002   10               Capitol Hill - Downtown Seattle                   Atlantic Base Via Broadway                         1            20010004      10   
                                                                            Capitol Hill Via 15th Ave E                        0            11010005       8   
                                                                                                                                            11010006     470   
                                                                                                                                            11010010       8   
                                                                            Downtown Seattle                                   1            20010002     470   
                                                                                                                                            20010007       2   
100003   101              Renton Transit Center - Downtown Seattle          Downtown Seattle                                   1            10101006     114   
                                                                                                                                            10101008     114   
                                                                            S Renton P&R                                       0            21101005     121   
                                                                                                                                            21101007     121   
100004   105              Renton Highlands - Renton Transit Center          Renton Highlands                                   0            11105002     168   
                                                                            Renton Renton Tech Coll                            1            20105003     174   
100005   106              Renton Transit Center - Skyway - Downtown Seattle International District Chinatown Station           1            10106037       6   
                                                                            International District Chinatown Station Rainie... 1            10106034     321   
                                                                            Renton Rainier Beach Station                       0            21106108     334   
100006   107              Renton Transit Center - Rainier Beach             Beacon Hill Station Rainier Beach Station          1            10107018     246   

                                                                                                                                                                \
                                                                                                                                                           min   
route_id route_short_name route_desc                                        trip_headsign                                      direction_id shap

### Ok, I think the following dataframe looks good for the final version

In [111]:
gtfs_aug18.trips.merge(
    gtfs_aug18.routes[['route_id', 'route_short_name', 'route_desc']],
    on='route_id'
).groupby(
    by=['route_short_name', 'direction_id']
).agg(
    {'route_desc': 'max', 
     'trip_headsign': lambda x: x.unique(),
     'shape_id': lambda x: len(x.unique()),
     'trip_id': 'count'}
).rename(
    columns={'shape_id': 'shape_count', 'trip_id': 'trip_count'}
        ).head(20)#.loc[('10',0), 'trip_headsign']
# .reset_index().set_index('route_short_name').head(20)

route_desc  \
route_short_name direction_id                                                      
1                0                                    Kinnear - Downtown Seattle   
                 1                                    Kinnear - Downtown Seattle   
10               0                               Capitol Hill - Downtown Seattle   
                 1                               Capitol Hill - Downtown Seattle   
101              0                      Renton Transit Center - Downtown Seattle   
                 1                      Renton Transit Center - Downtown Seattle   
102              0                                   Fairwood - Downtown Seattle   
                 1                                   Fairwood - Downtown Seattle   
105              0                      Renton Highlands - Renton Transit Center   
                 1                      Renton Highlands - Renton Transit Center   
106              0             Renton Transit Center - Skyway - Downtown Seattle   
                 1             Renton Transit Center - Skyway - Downtown Seattle   
107              0                         Renton Transit Center - Rainier Beach   
                 1                         Renton Transit Center - Rainier Beach   
11               0                Madison Park - Capitol Hill - Downtown Seattle   
                 1                Madison Park - Capitol Hill - Downtown Seattle   
111              0                              Lake Kathleen - Downtown Seattle   
                 1                              Lake Kathleen - Downtown Seattle   
113              0                                  Shorewood - Downtown Seattle   
                 1                                  Shorewood - Downtown Seattle   

                                                                   trip_headsign  \
route_short_name direction_id                                                      
1                0                                      Kinnear Seattle Center W   
                 1                                             Downtown Seattle    
10               0                                   Capitol Hill Via 15th Ave E   
                 1               [Downtown Seattle , Atlantic Base Via Broadway]   
101              0                                                 S Renton P&R    
                 1                                             Downtown Seattle    
102              0                        [Fairwood S Renton P&R, S Renton P&R ]   
                 1             [Downtown Seattle S Renton P&R, Downtown Seatt...   
105              0                                             Renton Highlands    
                 1                                       Renton Renton Tech Coll   
106              0                                  Renton Rainier Beach Station   
                 1             [International District Chinatown Station Rain...   
107              0                   Renton Transit Center Rainier Beach Station   
                 1                     Beacon Hill Station Rainier Beach Station   
11               0                                 Madison Park Via E Madison St   
                 1             [Downtown Seattle Via E Madison St, Stadium St...   
111              0                                        Lake Kathleen Via I-90   
                 1                                             Downtown Seattle    
113              0                                        Shorewood White Center   
                 1             [Downtown Seattle White Center, White Center S...   

                               shape_count  trip_count  
route_short_name direction_id                           
1                0                       2         282  
                 1                       2         282  
10               0                       3         486  
                 1                       3         482  
101              0            

In [110]:
# Hmm, using lambda x: x.unique() in .agg() returns a string if there's one object,
# and a numpy array of strings if there are more... Seems like that could be either useful or annoying...
# (Result of running the above cell with .loc[('10',0), 'trip_headsign'] uncommented)
type(_)

str

## Write a function to return a summary dataframe like the one above, optionally passing in specific route names

Note that the `Redmond LOOP` line only has one direction. The length of the returned dataframe for all routes is 438, which is 4 less than twice the number of routes (221), presumably because some routes only have one direction, like the Redmond Loop.

In [210]:
gtfs_aug18.trips_by_route_and_direction(10, 7, 'C Line', 'E Line', 8, 'Redmond LOOP', 41)

route_desc  \
route_short_name direction_id                                                     
10               0                              Capitol Hill - Downtown Seattle   
                 1                              Capitol Hill - Downtown Seattle   
41               0                  Lake City - Northgate TC - Downtown Seattle   
                 1                  Lake City - Northgate TC - Downtown Seattle   
7                0               Prentice St - Rainier Beach - Downtown Seattle   
                 1               Prentice St - Rainier Beach - Downtown Seattle   
8                0                Seattle Center - Capitol Hill - Rainier Beach   
                 1                Seattle Center - Capitol Hill - Rainier Beach   
C Line           0              Valley St/Fairview Av N - SW Barton St/26 Av SW   
                 1              Valley St/Fairview Av N - SW Barton St/26 Av SW   
E Line           0             Aurora Village Transit Center - Downtown Seattle   
                 1             Aurora Village Transit Center - Downtown Seattle   
Redmond LOOP     0                                                 Redmond Loop   

                                                                   trip_headsign  \
route_short_name direction_id                                                      
10               0                                   Capitol Hill Via 15th Ave E   
                 1               [Downtown Seattle , Atlantic Base Via Broadway]   
41               0             [Lake City Northgate, Northgate Northeast 125t...   
                 1               [Downtown Seattle , Downtown Seattle Northgate]   
7                0             [Rainier Beach Via Rainier Ave S, Prentice St ...   
                 1             [Downtown Seattle Via Rainier Ave S, S Jackson...   
8                0                                               Seattle Center    
                 1                                   Mount Baker Transit Center    
C Line           0             [Westwood Village , West Seattle Alaska Junction]   
                 1                             South Lake Union Downtown Seattle   
E Line           0                                Aurora Village Transit Center    
                 1             [Downtown Seattle , Stadium Station Downtown S...   
Redmond LOOP     0                                                 Redmond Loop    

                               shape_count  trip_count  block_count  \
route_short_name direction_id                                         
10               0                       3         486           36   
                 1                       3         482           36   
41               0                       8         450          110   
                 1                      10         438          102   
7                0                       7         641          137   
                 1                       5         641          137   
8                0                       1         388           67   
                 1                       1         396           67   
C Line           0                       4         579          134   
                 1                       2         581          148   
E Line           0                       1         551          152   
                 1                       2         554          153   
Redmond LOOP     0                       1          18            2   

                              trip_short_name peak_flag                fare_id  
route_short_name direction_id                                                   
10               0                      LOCAL    [0, 1]         [101.0, 110.0]  
                 1                      LOCAL    [0, 1]         [101.0, 110.0]  
41               0                      LOCAL    [0, 1]         [101.0, 110.0]  
                 1                      LOCAL    [0, 1]         [101.0, 110.0]  
7         

In [207]:
gtfs_aug18.trips_by_route_and_direction().shape

(438, 8)

In [217]:
trips_by_direction = gtfs_aug18.trips_by_route_and_direction()
trips_by_direction.loc['101', 'trip_short_name'] == 'LOCAL'
# trips_by_direction[(trips_by_direction.trip_short_name == 'LOCAL')]

direction_id
0    True
1    True
Name: trip_short_name, dtype: bool

In [198]:
gtfs_aug18.trips[gtfs_aug18.trips['fare_id'].isna()].shape

(2027, 10)

In [200]:
gtfs_aug18.trips['fare_id'].unique()

array([101., 110., 100., 501., 305., 900.,  nan, 906., 301., 102., 112.])

In [192]:
len(gtfs_aug18.trips['fare_id'].unique())

11

In [300]:
gtfs_aug18.trips['fare_id'].nunique() # .nunique() omits nan's by default

10

In [193]:
gtfs_aug18.trips['fare_id'].unique()[6]

nan

In [194]:
# WTF?! Why is this False??
gtfs_aug18.trips['fare_id'].unique()[6] == np.nan

False

In [203]:
# Oh, you need to use np.isnan
np.isnan(gtfs_aug18.trips['fare_id'].unique()[6])

True

In [299]:
set(gtfs_aug18.trips['fare_id'].head())

{101.0}

In [ ]:
# Old version of `list_unique` function, using set() instead of .unique()
# Results in multiple nan's rather than just one
# def list_unique(hashable_objects): return tuple(set(hashable_objects))

In [297]:
gt.list_unique(gtfs_aug18.trips['fare_id'])[:40] # Hmm, set() thinks the different nan's are distinct...

(nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 100.0,
 101.0,
 102.0,
 110.0,
 112.0,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan)

### Double check that our `trip_count` column is counting the right thing

The `count` function will count non-null rows, so our implementation assumes that each row in the trips table represents a unique trip. Let's verify this:

In [130]:
# Count total number of rows in trips table
gtfs_aug18.trips.shape

(59857, 10)

In [125]:
# Count number of unique trip_id's in trips table -- good, it's equal to above
gtfs_aug18.trips['trip_id'].unique().shape

(59857,)

In [123]:
# Make sure our trip counts add up to the right thing
gtfs_aug18.trips_by_route_and_direction()['trip_count'].sum()

59857

## Add another convenience function for the routes table

In [312]:
gtfs_aug18.routes_by_name(7, 120, 'D Line')

,route_desc,agency_id,route_id
route_short_name,,,
120,Burien TC - Westwood Village - Downtown Seattle,KCM,100019
7,Prentice St - Rainier Beach - Downtown Seattle,KCM,100263
D Line,Blue Ridge/Crown Hill - Ballard - Downtown Sea...,KCM,102581


In [148]:
# If we want a dictionary from route names to route id's. Leave out the route names if you want all routes.
gtfs_aug18.routes_by_name(7, 120, 'D Line')['route_id'].to_dict()

{'120': 100019, '7': 100263, 'D Line': 102581}

### Look at the entire fare attributes table since it's small

Best to keep unique fare id's rather than fare descriptions in the summary tables below, since two id's can have the same description.

In [316]:
gtfs_aug18.fare_attributes

,fare_id,agency_id,fare_period_id,price,descriptions,currency_type,payment_method,transfers,transfer_duration
0,100,KCM,1,0.00,KCM Regular Free,USD,0,NaN,5400
1,101,KCM,1,2.75,KCM Regular 1 Zone,USD,0,NaN,5400
2,102,KCM,1,2.75,KCM Regular 2 Zone,USD,0,NaN,5400
3,110,KCM,2,2.75,KCM Peak 1 Zone,USD,0,NaN,5400
4,112,KCM,2,2.75,KCM Peak 2 Zone,USD,0,NaN,5400
5,301,KCM,1,4.25,Boeing,USD,0,0.0,0
6,302,KCM,1,3.75,Boeing,USD,0,0.0,0
7,305,KCM,1,4.75,Custom High,USD,0,0.0,0
8,501,ST,1,2.75,ST Regular 1 Zone,USD,0,0.0,0
9,502,ST,1,3.75,ST Regular 2 Zone,USD,0,0.0,0


## Add function to group by route and shape

Whoa, what's the deal with all the shapes for the 41?

In [209]:
gtfs_aug18.trips_by_route_and_shape(7,8, 41, 'D Line')

direction_id                       trip_headsign  \
route_short_name shape_id                                                     
41               11041008             0                 Lake City Northgate   
                 11041011             0                 Lake City Northgate   
                 11041013             0      Northgate Northeast 125th Only   
                 11041015             0                 Lake City Northgate   
                 11041018             0                 Lake City Northgate   
                 11041020             0                 Lake City Northgate   
                 11041021             0      Northgate Northeast 125th Only   
                 11041023             0                 Lake City Northgate   
                 20041006             1          Downtown Seattle Northgate   
                 20041009             1                   Downtown Seattle    
                 20041010             1                   Downtown Seattle    
                 20041012             1          Downtown Seattle Northgate   
                 20041014             1          Downtown Seattle Northgate   
                 20041016             1          Downtown Seattle Northgate   
                 20041017             1                   Downtown Seattle    
                 20041019             1          Downtown Seattle Northgate   
                 20041022             1          Downtown Seattle Northgate   
                 20041024             1          Downtown Seattle Northgate   
7                10007006             1  Downtown Seattle Via Rainier Ave S   
                 10007007             1  Downtown Seattle Via Rainier Ave S   
                 10007012             1                       S Jackson St    
                 10007014             1  Downtown Seattle Via Rainier Ave S   
                 10007018             1  Downtown Seattle Via Rainier Ave S   
                 21007002             0     Rainier Beach Via Rainier Ave S   
                 21007003             0     Rainier Beach Via Rainier Ave S   
                 21007008             0     Rainier Beach Via Rainier Ave S   
                 21007013             0       Prentice St Via Rainier Ave S   
                 21007015             0       Prentice St Via Rainier Ave S   
                 21007016             0       Prentice St Via Rainier Ave S   
                 21007017             0       Prentice St Via Rainier Ave S   
8                31008004             0                     Seattle Center    
                 40008003             1         Mount Baker Transit Center    
D Line           11674004             0                      Ballard Uptown   
                 20674043             1             Downtown Seattle Uptown   
                 20674045             1    Stadium Station Downtown Seattle   

                           trip_count  block_count trip_short_name peak_flag  \
route_short_name shape_id                                                      
41               11041008          83           37           LOCAL    [0, 1]   
                 11041011         117           37           LOCAL    [0, 1]   
                 11041013          23           19           LOCAL    [0, 1]   
                 11041015           5            5           LOCAL       [0]   
                 11041018          80           32           LOCAL       [0]   
                 11041020         114           35           LOCAL       [0]   
                 11041021          23           19           LOCAL       [0]   
                 11041023           5            5           LOCAL       [0]   
                 20041006         133           38           LOCAL    [0, 1]   
                 20041009          11           11           LOCAL    [0, 1]   
                 20041010           6            6           LOCAL    [0, 1]   
                 20041012          70           36           LO

### Hmm, some of the shape id's changed between July and August

For the 41, there are 18 shapes for both months

In [218]:
gtfs_jul18.trips_by_route_and_shape(7,8, 41, 'D Line')

direction_id                       trip_headsign  \
route_short_name shape_id                                                     
41               11041002             0                 Lake City Northgate   
                 11041008             0                 Lake City Northgate   
                 11041011             0                 Lake City Northgate   
                 11041013             0      Northgate Northeast 125th Only   
                 11041015             0                 Lake City Northgate   
                 11041231             0                 Lake City Northgate   
                 11041232             0                 Lake City Northgate   
                 11041240             0      Northgate Northeast 125th Only   
                 20041003             1                   Downtown Seattle    
                 20041005             1          Downtown Seattle Northgate   
                 20041006             1          Downtown Seattle Northgate   
                 20041009             1                   Downtown Seattle    
                 20041010             1                   Downtown Seattle    
                 20041012             1          Downtown Seattle Northgate   
                 20041014             1          Downtown Seattle Northgate   
                 20041228             1          Downtown Seattle Northgate   
                 20041229             1                   Downtown Seattle    
                 20041250             1          Downtown Seattle Northgate   
7                10007006             1  Downtown Seattle Via Rainier Ave S   
                 10007007             1  Downtown Seattle Via Rainier Ave S   
                 10007012             1                       S Jackson St    
                 10007014             1  Downtown Seattle Via Rainier Ave S   
                 10007018             1  Downtown Seattle Via Rainier Ave S   
                 21007002             0     Rainier Beach Via Rainier Ave S   
                 21007003             0     Rainier Beach Via Rainier Ave S   
                 21007008             0     Rainier Beach Via Rainier Ave S   
                 21007013             0       Prentice St Via Rainier Ave S   
                 21007015             0       Prentice St Via Rainier Ave S   
                 21007016             0       Prentice St Via Rainier Ave S   
                 21007017             0       Prentice St Via Rainier Ave S   
8                31008004             0                     Seattle Center    
                 40008003             1         Mount Baker Transit Center    
D Line           11674004             0                      Ballard Uptown   
                 20674043             1             Downtown Seattle Uptown   
                 20674045             1    Stadium Station Downtown Seattle   

                           trip_count  block_count trip_short_name peak_flag  \
route_short_name shape_id                                                      
41               11041002           5            5           LOCAL       [0]   
                 11041008          83           37           LOCAL    [0, 1]   
                 11041011         117           37           LOCAL    [0, 1]   
                 11041013          23           19           LOCAL    [0, 1]   
                 11041015           5            5           LOCAL       [0]   
                 11041231         114           35           LOCAL       [0]   
                 11041232          80           32           LOCAL       [0]   
                 11041240          23           19           LOCAL       [0]   
                 20041003           6            5           LOCAL       [0]   
                 20041005         128           34           LOCAL       [0]   
                 20041006         133           38           LOCAL    [0, 1]   
                 20041009          11           11           LO

## For some reason the following line originally threw an error

Mon-Wed, March 25-27, 2019

When calling `gtfs_jul18.trips_by_route_and_direction(41)` below, I got the error message "ValueError: Function does not reduce", even though the function seemed to be working fine above. After googling it, the problem [seems to be](https://stackoverflow.com/questions/27439023/pandas-groupby-agg-function-does-not-reduce) that pandas doesn't like it when either (1) an aggregation function can return different types (e.g. int or list), or (2) the returned types are not hashable (e.g. a list vs. a tuple).

The recommended solution was to always return a tuple, which is immutable and hence hashable, if you want to aggregate into a list. (Strangely, calling `.nunique()` sometimes gave the same "Function does not reduce" error as `.unique()`, even though it should always just return a number, not a numpy array. I think the problem has to do with the `nan`'s.)

I wrote a collection of simple aggregation functions such as `def list_unique(hashable_objects): return tuple(set(hashable_objects))` and used these instead of calling `.unique()` or `.nunique()`. So far, this seems to work...

Wed 3/27/19: Hmm... calling `set()` on a Series object with `nan`'s seems to retain multiple `nan`'s rather than mapping them all to a single `nan` (see example above), so maybe it's best to use pandas' `.unique()` function instead. I changed the above function to `def list_unique(hashable_objects: pd.Series) -> tuple: return tuple(hashable_objects.unique())`. This seems to still be working so far...

In [317]:
gtfs_jul18.trips_by_route_and_direction(41)

route_desc  \
route_short_name direction_id                                                
41               0             Lake City - Northgate TC - Downtown Seattle   
                 1             Lake City - Northgate TC - Downtown Seattle   

                                                                   trip_headsign  \
route_short_name direction_id                                                      
41               0             (Lake City Northgate, Northgate Northeast 125t...   
                 1               (Downtown Seattle , Downtown Seattle Northgate)   

                               shape_count  trip_count  block_count  \
route_short_name direction_id                                         
41               0                       8         450          110   
                 1                      10         438          102   

                              trip_short_name peak_flag         fare_id  
route_short_name direction_id                                            
41               0                   (LOCAL,)    (0, 1)  (101.0, 110.0)  
                 1                   (LOCAL,)    (0, 1)  (101.0, 110.0)

In [318]:
# WTF?! Why does this work now even though I haven't updated the aggregation functions?
gtfs_jul18.trips_by_route_and_shape(41)

direction_id                      trip_headsign  \
route_short_name shape_id                                                    
41               11041002             0             (Lake City Northgate,)   
                 11041008             0             (Lake City Northgate,)   
                 11041011             0             (Lake City Northgate,)   
                 11041013             0  (Northgate Northeast 125th Only,)   
                 11041015             0             (Lake City Northgate,)   
                 11041231             0             (Lake City Northgate,)   
                 11041232             0             (Lake City Northgate,)   
                 11041240             0  (Northgate Northeast 125th Only,)   
                 20041003             1               (Downtown Seattle ,)   
                 20041005             1      (Downtown Seattle Northgate,)   
                 20041006             1      (Downtown Seattle Northgate,)   
                 20041009             1               (Downtown Seattle ,)   
                 20041010             1               (Downtown Seattle ,)   
                 20041012             1      (Downtown Seattle Northgate,)   
                 20041014             1      (Downtown Seattle Northgate,)   
                 20041228             1      (Downtown Seattle Northgate,)   
                 20041229             1               (Downtown Seattle ,)   
                 20041250             1      (Downtown Seattle Northgate,)   

                           trip_count  block_count trip_short_name peak_flag  \
route_short_name shape_id                                                      
41               11041002           5            5        (LOCAL,)      (0,)   
                 11041008          83           37        (LOCAL,)    (0, 1)   
                 11041011         117           37        (LOCAL,)    (0, 1)   
                 11041013          23           19        (LOCAL,)    (0, 1)   
                 11041015           5            5        (LOCAL,)      (0,)   
                 11041231         114           35        (LOCAL,)      (0,)   
                 11041232          80           32        (LOCAL,)      (0,)   
                 11041240          23           19        (LOCAL,)      (0,)   
                 20041003           6            5        (LOCAL,)      (0,)   
                 20041005         128           34        (LOCAL,)      (0,)   
                 20041006         133           38        (LOCAL,)    (0, 1)   
                 20041009          11           11        (LOCAL,)    (0, 1)   
                 20041010           6            6        (LOCAL,)    (0, 1)   
                 20041012          70           36        (LOCAL,)    (0, 1)   
                 20041014           2            2        (LOCAL,)      (0,)   
                 20041228          69           32        (LOCAL,)      (0,)   
                 20041229          11           11        (LOCAL,)      (0,)   
                 20041250           2            2        (LOCAL,)      (0,)   

                                  fare_id  
route_short_name shape_id                  
41               11041002        (101.0,)  
                 11041008  (101.0, 110.0)  
                 11041011  (101.0, 110.0)  
                 11041013  (101.0, 110.0)  
                 11041015        (101.0,)  
                 11041231        (101.0,)  
                 11041232        (101.0,)  
                 11041240        (101.0,)  
                 20041003        (101.0,)  
                 20041005        (101.0,)  
                 20041006  (101.0, 110.0)  
                 20041009  (101.0, 110.0)  
                 20041010  (101.0, 110.0)  
                 20041012  (101.0, 110.0)  
                 20041014        (101.0,)  
                 20041228        (101.0,)  
                 20041229        (101.0,)  
                 20041250    

In [277]:
gtfs_jul18.route_ids()['41']
# gtfs_jul18.trips.set_index('route_id').loc[gtfs_jul18.route_ids(41, 'C Line')]

100221

In [273]:
# gtfs_jul18.routes_by_name(41)['route_id'].to_dict()['41']
# gtfs_jul18.routes_by_name().loc['41', 'route_id']
# jul_trips_41 = gtfs_jul18.trips.set_index('route_id').loc[gtfs_jul18.routes_by_name().loc['41', 'route_id']]
jul_trips_41 = gtfs_jul18.trips.set_index('route_id').loc[gtfs_jul18.route_ids(41)]
jul_trips_41.head(10)

,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,peak_flag,fare_id
route_id,,,,,,,,,
100221,44711,39485224,Downtown Seattle,LOCAL,1,5062060,20041003,0,101.0
100221,44711,39485225,Lake City Northgate,LOCAL,0,5061877,11041232,0,101.0
100221,44711,39485226,Downtown Seattle Northgate,LOCAL,1,5061877,20041228,0,101.0
100221,44711,39485777,Lake City Northgate,LOCAL,0,5061877,11041232,0,101.0
100221,44711,39485778,Downtown Seattle Northgate,LOCAL,1,5061873,20041005,0,101.0
100221,44711,39485779,Downtown Seattle Northgate,LOCAL,1,5061872,20041005,0,101.0
100221,44711,39485780,Downtown Seattle Northgate,LOCAL,1,5061871,20041005,0,101.0
100221,44711,39485781,Lake City Northgate,LOCAL,0,5061871,11041231,0,101.0
100221,44711,39485782,Downtown Seattle Northgate,LOCAL,1,5061880,20041228,0,101.0


In [274]:
jul_trips_41.shape

(888, 9)

In [242]:
jul_trips_41.fare_id.unique()

array([101., 110.])

In [243]:
jul_trips_41.fare_id.nunique()

2

In [249]:
jul_trips_41.groupby('direction_id').agg({'fare_id': lambda x: x.nunique()})

,fare_id
direction_id,
0,2.0
1,2.0


In [287]:
# Test what happens when you sort non-comparable objects.
# You get: "TypeError: '<' not supported between instances of 'StaticGTFS' and 'StaticGTFS'"
# sorted([gtfs_aug18, gtfs_jul18, gtfs_aug18])

TypeError: '<' not supported between instances of 'StaticGTFS' and 'StaticGTFS'

## Check out the `stops` and `stop_times` tables

Still working on simplifying the "find the closest stop to the bus" process from Andrew's nobebook 2 and my notebook 4, "transofm E locations" and "transform D locations".

In [320]:
gtfs_jul18.stops.shape

(7686, 11)

In [322]:
# The stop times table is huge!
gtfs_jul18.stop_times.shape

(2062501, 9)

### July 2018 `stops` table

In [323]:
gtfs_jul18.stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone
0,1000,NaN,Pine St & 9th Ave,NaN,47.613415,-122.332138,21,NaN,0,NaN,America/Los_Angeles
1,10000,NaN,NE 55th St & 43rd Ave NE,NaN,47.668575,-122.283653,1,NaN,0,NaN,America/Los_Angeles
2,10005,NaN,40th Ave NE & NE 51st St,NaN,47.665886,-122.284897,1,NaN,0,NaN,America/Los_Angeles
3,10010,NaN,NE 55th St & 39th Ave NE,NaN,47.668579,-122.285667,1,NaN,0,NaN,America/Los_Angeles
4,10020,NaN,NE 55th St & 37th Ave NE,NaN,47.668579,-122.288300,1,NaN,0,NaN,America/Los_Angeles


In [330]:
gtfs_jul18.stops['stop_id'].nunique() # The stops table is indexed by stop_id

7686

In [343]:
gtfs_jul18.stops['stop_name'].nunique() # Hmm, there are 1500 fewer stop names than id's?

5193

In [345]:
gtfs_jul18.stops[gtfs_jul18.stops['stop_name'].isna()] # So the difference doesn't seem to be null values...

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone


### Question: How can we find duplicate names in the stops table?

It might be nice to have a function/Series that mapps stop names to stop ids, like I did for routes, but it appears that not every name corresponds to a unique id. I'd like to find the duplicate names to see whether such a mapping would make sense. Perhaps you can do it using a self join (aka merge)?

(See below for answer -- simply re-index the table by stop name.)

### But first, see what the `stop_times` table looks like:

In [324]:
gtfs_jul18.stop_times.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
0,34734386,07:03:00,07:03:00,13390,1,NaN,0,0,0.0
1,34734386,07:04:11,07:04:11,13410,4,NaN,0,0,1130.9
2,34734386,07:05:10,07:05:10,13420,9,NaN,0,0,2057.1
3,34734386,07:06:00,07:06:00,13430,14,NaN,0,0,2833.3
4,34734386,07:06:42,07:06:42,13441,18,NaN,0,0,3491.2


### It looks like several columns  in the `stops` table contain no useful data - let's double check

In [331]:
print(gtfs_jul18.stops['stop_code'].unique(),
      gtfs_jul18.stops['stop_desc'].unique(),
      gtfs_jul18.stops['stop_url'].unique(),
      gtfs_jul18.stops['location_type'].unique(),
      gtfs_jul18.stops['parent_station'].unique(),
      gtfs_jul18.stops['stop_timezone'].unique()
     )

[nan] [nan] [nan] [0] [nan] ['America/Los_Angeles']


In [328]:
gtfs_aug18.stops['location_type'].unique()

array([0])

In [332]:
gtfs_jul18.stops['zone_id'].unique()

array([21,  1, 18, 20,  3,  6, 24, 14,  5, 26, 22, 74, 40,  9, 39,  4, 19,
        8, 12, 78, 77, 76, 75, 79, 36, 71, 17, 72, 73, 82, 81, 80])

### Write a function to extract the useful columns from stops table and index it by `stop_id`

Rather than passing a variable length list of arguments, like I did with route names, this function assumes you pass either a single id or a `list` (or other iterable) of stop id's. Because for id's you're more likely to extract them programmatically from somewhere else, rather than typing them in directly like you would for names.

When the single `stop_ids` argument to the `stops_by_id()` function is not `None`, the behavior of the returned object is inhereted from pandas indexing.

In [342]:
gtfs_aug18.stops_by_id().head() #Passing no arguments returns all stops

,stop_name,zone_id,stop_lat,stop_lon
stop_id,,,,
1000,Pine St & 9th Ave,21,47.613415,-122.332138
10000,NE 55th St & 43rd Ave NE,1,47.668575,-122.283653
10005,40th Ave NE & NE 51st St,1,47.665886,-122.284897
10010,NE 55th St & 39th Ave NE,1,47.668579,-122.285667
10020,NE 55th St & 37th Ave NE,1,47.668579,-122.288300


In [346]:
gtfs_aug18.stops_by_id(1000) #Passing a single id returns one row (Series object)

stop_name    Pine St & 9th Ave
zone_id                     21
stop_lat               47.6134
stop_lon              -122.332
Name: 1000, dtype: object

In [348]:
gtfs_aug18.stops_by_id([1000]) #Passing a list of ids returns a sub-dataframe

,stop_name,zone_id,stop_lat,stop_lon
stop_id,,,,
1000,Pine St & 9th Ave,21,47.613415,-122.332138


In [347]:
gtfs_aug18.stops_by_id([1000,10000]) #Passing a list of ids returns a sub-dataframe

,stop_name,zone_id,stop_lat,stop_lon
stop_id,,,,
1000,Pine St & 9th Ave,21,47.613415,-122.332138
10000,NE 55th St & 43rd Ave NE,1,47.668575,-122.283653


### Answer my above question about finding stop ids that have the same stop name

So it could still be useful to write a function that returns all stop id's for a given name.

In [352]:
# Re-index the stops table by the name column to group stops with the same name
# If we wanted to find only names that are duplicated, we could group by stop_name
# and aggregate stop_id by count, then order by the count.
gtfs_aug18.stops.set_index('stop_name').loc[gtfs_aug18.stops['stop_name']].head(20)

,stop_id,stop_code,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,stop_timezone
stop_name,,,,,,,,,,
Pine St & 9th Ave,1000,NaN,NaN,47.613415,-122.332138,21,NaN,0,NaN,America/Los_Angeles
Pine St & 9th Ave,1085,NaN,NaN,47.613377,-122.332603,21,NaN,0,NaN,America/Los_Angeles
NE 55th St & 43rd Ave NE,10000,NaN,NaN,47.668575,-122.283653,1,NaN,0,NaN,America/Los_Angeles
40th Ave NE & NE 51st St,10005,NaN,NaN,47.665886,-122.284897,1,NaN,0,NaN,America/Los_Angeles
NE 55th St & 39th Ave NE,10010,NaN,NaN,47.668579,-122.285667,1,NaN,0,NaN,America/Los_Angeles
NE 55th St & 37th Ave NE,10020,NaN,NaN,47.668579,-122.288300,1,NaN,0,NaN,America/Los_Angeles
NE 55th St & 35th Ave NE,10030,NaN,NaN,47.668579,-122.290512,1,NaN,0,NaN,America/Los_Angeles
NE 55th St & 35th Ave NE,10500,NaN,NaN,47.668446,-122.290390,1,NaN,0,NaN,America/Los_Angeles
NE 55th St & 33rd Ave NE,10040,NaN,NaN,47.668583,-122.293015,1,NaN,0,NaN,America/Los_Angeles


## Write a function to return all the trips for a given route and direction

In [367]:
gtfs_jul18.route_ids(8)['8']

100275

In [380]:
gtfs_jul18.trips_for_route_and_direction(8,1).head()

,route_short_name,route_id,service_id,trip_id,trip_headsign,trip_short_name,direction_id,block_id,shape_id,peak_flag,fare_id
0,8,100275,44711,39432385,Mount Baker Transit Center,LOCAL,1,5051095,40008003,0,101.0
1,8,100275,44711,39432386,Mount Baker Transit Center,LOCAL,1,5051093,40008003,0,101.0
2,8,100275,44711,39432388,Mount Baker Transit Center,LOCAL,1,5051094,40008003,0,101.0
3,8,100275,44711,39432389,Mount Baker Transit Center,LOCAL,1,5051096,40008003,0,101.0
4,8,100275,44711,39432390,Mount Baker Transit Center,LOCAL,1,5051097,40008003,0,101.0


In [388]:
gtfs_jul18.trip_ids_for_route_and_direction(8,1).head() #Hmm, do we want to keep the original indices or reset?

23464    39432385
23465    39432386
23467    39432388
23468    39432389
23469    39432390
Name: trip_id, dtype: int64

In [389]:
gtfs_jul18.trip_ids_for_route_and_direction(8,1).head().values

array([39432385, 39432386, 39432388, 39432389, 39432390])

In [395]:
#trip_ids = gtfs_jul18.trip_ids_for_route_and_direction(8,1).head()
gtfs_jul18.stop_times.set_index('trip_id')[39432386]#.head()#[trip_ids]

KeyError: 39432386